<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/LSTM_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -U imbalanced-learn

In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 403 ms, sys: 72.9 ms, total: 475 ms
Wall time: 521 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual,NaN
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual,NaN
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous,"fit small size , size large normal available s..."
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
X = tokenizer.texts_to_sequences(X)

In [0]:
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [15]:
def load_glove_vectors():
    embeddings_index = {}
    with open('/content/drive/My Drive/NLP/glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    #model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = define_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 100)          611400    
_________________________________________________________________
masking_1 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 155, 64)           42240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 11)                187       
Total params: 687,891
Trainable params: 76,491
Non-trainable params: 611,400
___________________________________________

In [19]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 20382 samples, validate on 5096 samples
Epoch 1/10
20382/20382 [==============================] - 52s 3ms/step - loss: 0.6702 - accuracy: 0.8151 - val_loss: 0.5226 - val_accuracy: 0.8508
Epoch 2/10
20382/20382 [==============================] - 51s 2ms/step - loss: 2.4581 - accuracy: 0.8839 - val_loss: 0.4649 - val_accuracy: 0.9091
Epoch 3/10
20382/20382 [==============================] - 51s 3ms/step - loss: 0.4222 - accuracy: 0.9090 - val_loss: 0.4088 - val_accuracy: 0.9091
Epoch 4/10
20382/20382 [==============================] - 51s 2ms/step - loss: 0.3727 - accuracy: 0.9091 - val_loss: 0.3857 - val_accuracy: 0.9091
Epoch 5/10
20382/20382 [==============================] - 50s 2ms/step - loss: 0.3470 - accuracy: 0.9091 - val_loss: 0.3824 - val_accuracy: 0.9091
Epoch 6/10
20382/20382 [==============================] - 50s 2ms/step - loss: 0.3337 - accuracy: 0.9091 - val_loss: 0.3886 - val_accuracy: 0.9091
Epoch 7/10
20382/20382 [==============================] - 51s 2ms/ste

In [20]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 6s 5ms/step


[0.3227814110842618, 0.909090518951416]

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [23]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [24]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, businesscasual,..."
1,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, businesscasual,..."
2,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, businesscasual,..."
3,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, businesscasual,..."
4,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,,,"androgynous, athleisure, boho, businesscasual,..."


In [25]:
results_df['attribute_value'].value_counts()

androgynous, athleisure, boho, businesscasual, casual, classic, edgy, glam, modern                     8142
androgynous, athleisure, boho, businesscasual, casual, classic, edgy, glam, modern, retro, romantic    1384
androgynous, athleisure, boho, businesscasual, casual, classic, edgy, glam                             1096
androgynous, athleisure, boho, businesscasual, casual, classic, edgy                                     86
athleisure, boho, businesscasual, casual, classic, glam                                                  50
athleisure, boho, glam                                                                                   20
athleisure, boho, businesscasual, casual, classic, edgy, glam                                            18
athleisure, boho, classic, glam                                                                          15
androgynous, athleisure, boho, businesscasual, casual, classic, glam                                     14
androgynous, athleisure, boh